# Lab 8 - BCC406

## REDES NEURAIS E APRENDIZAGEM EM PROFUNDIDADE

## LSTM e Transformers

### Prof. Eduardo e Prof. Pedro

Objetivos:

- Parte I : Uso de LSTM com uma camada

- Parte II : Uso de LSTM com duas camadas

- Parte III : Uso de LSTM com duas camadas bidirecionais

- Parte IV: Uso de Transformers


Data da entrega : 01/04

- Complete o código (marcado com ToDo) e quando requisitado, escreva textos diretamente nos notebooks. Onde tiver *None*, substitua pelo seu código.
- Execute todo notebook e salve tudo em um PDF **nomeado** como "NomeSobrenome-Lab.pdf"
- Envie o PDF via google [FORM](https://forms.gle/RXTxPuGuNevW4HgdA)

Este notebook é baseado em tensorflow e Keras.

# Representações de texto (*Bag-of-Words* vs. *Word Embeddings*) em NLP

Neste exercício prático, vamos explorar duas abordagens diferentes para representar textos em português e treinar um modelo de classificação de texto simples. Usaremos um dataset desafiador de NLP em português – por exemplo, **[resenhas de filmes](https://https://drive.google.com/file/d/1KVIxGF6AVD6i43JPT0DBIZzYKJrBMoE5/view?usp=drive_link)** traduzidas para PT-BR, rotuladas como _positivas_ ou _negativas_.  O [dataset](https://https://drive.google.com/file/d/1KVIxGF6AVD6i43JPT0DBIZzYKJrBMoE5/view?usp=drive_link) contempla o review dado pelos usuários e o sentimento daquele review (positivo/negativo) com relação ao filme. O objetivo é, dado um review (ou resenha) em portugues, classificar o texto como positivo ou negativo.

O exercício será dividido em duas etapas principais:  

1. **LSTM + MLP:** Uso de LSTM com uma única camada e uma MLP para classificação.  
2. **LSTMx2 + MLP:** Uso de duas camadas de LSTM e uma MLP para classificação.  
2. **LSTMx2B + MLP:** Uso de duas camadas de LSTM bidirecionais e uma MLP para classificação.  
2. **Transformers + MLP:** Uso do modelo Transformer, especificamente o BERTimbau, para realizar a classificação.

Para todas as quatro etapas, você pode usar (ou não) uma camada de **embedding** para converter palavras em vetores densos de dimensões menores.

## Importando as bibliotecas


Aqui, faremos a importação de todas as bibliotecas que serão usadas nesta prática.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import Trainer
from transformers import TrainingArguments

## Preparando os dados


Aqui assumiremos um dataset de **resenhas de filmes em português**, já categorizadas como sendo de sentimento **positivo** ou **negativo**.

O dataset está disponível em um arquivo CSV, [na pasta da prática](https://https://drive.google.com/file/d/1KVIxGF6AVD6i43JPT0DBIZzYKJrBMoE5/view?usp=drive_link). Os passos a seguir demonstram como carregar e inspecionar os dados.

O primeiro passo é fazer o download do arquivo.


In [ ]:
!gdown 1KVIxGF6AVD6i43JPT0DBIZzYKJrBMoE5

Downloading...
From (original): https://drive.google.com/uc?id=1KVIxGF6AVD6i43JPT0DBIZzYKJrBMoE5
From (redirected): https://drive.google.com/uc?id=1KVIxGF6AVD6i43JPT0DBIZzYKJrBMoE5&confirm=t&uuid=ee667479-fba4-4e49-9d18-1fefeed90ac3
To: /content/imdb-reviews-pt-br.csv
100% 127M/127M [00:02<00:00, 50.5MB/s]


Fazer a leitura do arquivo das resenhas com o pacote Pandas.

In [ ]:
df = pd.read_csv('imdb-reviews-pt-br.csv')

Inspecionando os dados.

In [ ]:
# Verificar as primeiras linhas do dataset para entender sua estrutura
print(f"Número de exemplos: {len(df)}")
print(df.head(5))

Número de exemplos: 49459
   id                                            text_en  \
0   1  Once again Mr. Costner has dragged out a movie...   
1   2  This is an example of why the majority of acti...   
2   3  First of all I hate those moronic rappers, who...   
3   4  Not even the Beatles could write songs everyon...   
4   5  Brass pictures movies is not a fitting word fo...   

                                             text_pt sentiment  
0  Mais uma vez, o Sr. Costner arrumou um filme p...       neg  
1  Este é um exemplo do motivo pelo qual a maiori...       neg  
2  Primeiro de tudo eu odeio esses raps imbecis, ...       neg  
3  Nem mesmo os Beatles puderam escrever músicas ...       neg  
4  Filmes de fotos de latão não é uma palavra apr...       neg  


**Explicação:** O código acima lê o arquivo CSV contendo as resenhas. Substitua `'imdb-reviews-pt-br.csv'` pelo caminho adequado do seu dataset. Usamos `df.head(5)` para ver as primeiras 5 entradas e inspecionar as colunas. Provavelmente, o dataset terá uma coluna para o texto da resenha (por exemplo, `review_pt` ou `texto`) e outra para o rótulo de sentimento (por exemplo, `sentiment` indicando *positivo/negativo*).  

O próximo passo é  extrair as colunas de texto e rótulo para listas (ou arrays) separados, o que facilitará o manuseio posteriormente.

In [ ]:
texts = df['text_pt'].astype(str).values    # convertendo para string por segurança
labels = df['sentiment'].map({'neg': 0, 'pos': 1}).values

Fazendo uma análise do que foi carregado.

In [ ]:
print("Total de textos:", len(texts))
print("Exemplo de texto:", texts[0][:100], "...")  # imprime começo do primeiro texto
print("Rótulo desse texto:", labels[0])

Total de textos: 49459
Exemplo de texto: Mais uma vez, o Sr. Costner arrumou um filme por muito mais tempo do que o necessário. Além das terr ...
Rótulo desse texto: 0


**Nota:** Caso seu dataset tenha rótulos como "positivo"/"negativo" ou "pos"/"neg", converta-os para valores numéricos (e.g., 1 para positivo, 0 para negativo) conforme mostrado no comentário acima, pois isso facilita o treinamento do modelo.  

Para simplificar o exercício e reduzir tempo de processamento (deixando-o _leve_), podemos **opcionalmente** trabalhar com uma amostra menor do dataset. Por exemplo, usar apenas 10.000 exemplos se o conjunto completo for muito grande.

In [ ]:
# OPCIONAL: usar somente uma parte dos dados para treinamento mais rápido (por exemplo, 10000 primeiras linhas)
df = df.sample(10000, random_state=42)  # amostra aleatória de 10000 exemplos

Agora que os dados estão carregados e prontos, vamos iniciar a **Parte 1: Bag-of-Words + MLP**.

In [ ]:
MAX_WORDS = 5000
EMBEDDING_DIM = 50
MAX_LEN = 100

In [ ]:
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X_seq = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    X_seq, labels, test_size=0.2, random_state=42)

## LSTM + MLP

Nesta parte, vamos implementar uma arquitetura com somente uma única camada (no mínimo 32 unidades) de LSTM e uma MLP para classificação.  

In [ ]:
modelo1 = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
    LSTM(32, return_sequences=False),
    Dense(128, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

modelo1.summary()
modelo1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

modelo1.fit(X_train_seq,
            y_train_seq,
            epochs=10,
            batch_size=64,
)

modelo1.evaluate(X_train_seq, y_train_seq)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.6489 - loss: 0.5916
Epoch 2/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8414 - loss: 0.3646
Epoch 3/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8615 - loss: 0.3187
Epoch 4/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8771 - loss: 0.2926
Epoch 5/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8899 - loss: 0.2636
Epoch 6/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9071 - loss: 0.2249
Epoch 7/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9219 - loss: 0.1970
Epoch 8/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9334 - loss: 0.1751
Epoch 9/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9432 - loss: 0.1551
Epoch 10/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9518 - loss: 0.1371
1237/1237 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9682 - loss: 0.1038


[0.10101693123579025, 0.9692673087120056]

## LSTMx2 + MLP

Nesta parte, vamos implementar uma arquitetura com duas camadas (no mínimo 32 unidades em cada uma) de LSTM e uma MLP para classificação.

In [ ]:
modelo2 = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
    LSTM(64, return_sequences=True),
    LSTM(32, return_sequences=False),
    Dense(128, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

modelo2.summary()
modelo2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

modelo2.fit(X_train_seq,
            y_train_seq,
            epochs=10,
            batch_size=64,
)

modelo2.evaluate(X_train_seq, y_train_seq)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.6723 - loss: 0.5876
Epoch 2/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.8308 - loss: 0.4007
Epoch 3/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.8540 - loss: 0.3462
Epoch 4/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.8757 - loss: 0.2975
Epoch 5/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.8984 - loss: 0.2554
Epoch 6/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9122 - loss: 0.2234
Epoch 7/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9285 - loss: 0.1903
Epoch 8/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9419 - loss: 0.1643
Epoch 9/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9481 - loss: 0.1484
Epoch 10/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9573 - loss: 0.1241
1237/1237 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9721 - loss: 0.0977


[0.09570758044719696, 0.9730836153030396]

## LSTMx2B + MLP

Nesta parte, vamos implementar uma arquitetura com duas camadas (no mínimo 32 unidades em cada uma) de LSTM bidirecional e uma MLP para classificação.  

In [ ]:
modelo3 = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32, return_sequences=False)),
    Dense(128, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

modelo3.summary()
modelo3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

modelo3.fit(X_train_seq,
            y_train_seq,
            epochs=10,
            batch_size=64,
)

modelo3.evaluate(X_train_seq, y_train_seq)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.6909 - loss: 0.5545
Epoch 2/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.8513 - loss: 0.3408
Epoch 3/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.8824 - loss: 0.2822
Epoch 4/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.9052 - loss: 0.2357
Epoch 5/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.9268 - loss: 0.1951
Epoch 6/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.9478 - loss: 0.1431
Epoch 7/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9625 - loss: 0.1058
Epoch 8/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.9722 - loss: 0.0833
Epoch 9/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.9798 - loss: 0.0603
Epoch 10/10
619/619 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.9825 - loss: 0.0543
1237/1237 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9900 - loss: 0.0332


[0.03272402659058571, 0.9903707504272461]

## Tranformers + MLP

Nesta parte, vamos implementar uma arquitetura utilizando o BERTimbau  e uma MLP para classificação. Para utilizar o BERTimbau no seu modelo, siga o tutorial neste [link](https://www.tensorflow.org/text/tutorials/classify_text_with_bert?hl=pt-br).

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
MODEL = "neuralmind/bert-large-portuguese-cased"
BATCH_SIZE= 8
NUM_LABELS = 2

In [ ]:
# Diminuição da quantidade de itens para conseguir rodar no Colab
# Antes o tempo estipulado para treinamento ultrapassava 18 horas

texts = texts[:500]
print(texts.shape)

labels = labels[:500]
print(labels.shape)

(500,)
(500,)


In [ ]:

def convert_to_huggingface(train_texts, train_labels, val_texts, val_labels, test_texts, test_labels):
    train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
    val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})
    test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})
    return DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})


def tokenize_function(example):
    return tokenizer(example['text'], padding="max_length", truncation=True, max_length=512)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}


X_temp, X_val, y_temp, y_val = train_test_split(texts, labels, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2)


dataset = convert_to_huggingface(
    train_texts=list(X_train), train_labels=list(y_train),
    val_texts=list(X_val), val_labels=list(y_val),
    test_texts=list(X_test), test_labels=list(y_test)
)

tokenizer = BertTokenizer.from_pretrained(MODEL)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=512)

train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['validation']

model = BertForSequenceClassification.from_pretrained(MODEL, num_labels=NUM_LABELS)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

# ToDo: Seu código aqui para validar a resposta do seu modelo
test_dataset = tokenized_datasets['test']
results = trainer.evaluate(test_dataset)
print(results)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000096,1.000000
2,No log,0.000052,1.000000
3,No log,0.000039,1.000000
4,No log,0.000033,1.000000
5,No log,0.000032,1.000000


{'eval_loss': 3.240055593778379e-05, 'eval_accuracy': 1.0, 'eval_runtime': 9.0384, 'eval_samples_per_second': 9.957, 'eval_steps_per_second': 1.328, 'epoch': 5.0}
